In [41]:
def g_0(n):
    return ("?",)*n

def s_0(n):
    return ('0',)*n

In [42]:
def more_general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)

In [43]:
def fulfills(example, hypothesis):
    return more_general(hypothesis, example)

def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not fulfills(x[i:i+1], h[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]

In [44]:
def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

In [45]:
import csv

# with open('enjoySport.csv')  as csvFile:
#         examples = [tuple(line)[1:] for line in csv.reader(csvFile)]
# examples=examples[1:]
with open('shapes.csv')  as csvFile:
        examples = [tuple(line)[1:] for line in csv.reader(csvFile)]
examples=examples[1:]
examples

[('Big', 'Red', 'Circle', 'N'),
 ('Small', 'Red', 'Triangle', 'N'),
 ('Small', 'Red', 'Circle', 'Y'),
 ('Big', 'Blue', 'Circle', 'N'),
 ('Small', 'Blue', 'Circle', 'Y')]

In [46]:
def get_domains(examples):
    d = [set() for i in examples[0]]
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]

get_domains(examples)

[['Big', 'Small'], ['Blue', 'Red'], ['Circle', 'Triangle'], ['N', 'Y']]

In [47]:
def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not fulfills(x, s):
            S.remove(s)
            Splus = min_generalizations(s, x)
            S.update([h for h in Splus if any([more_general(g,h) 
                                               for g in G])])
            S.difference_update([h for h in S if 
                                 any([more_general(h, h1) 
                                      for h1 in S if h != h1])])
    return S

In [48]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if fulfills(x, g):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            G.update([h for h in Gminus if any([more_general(h, s) for s in S])])
            G.difference_update([h for h in G if any([more_general(g1, h) for g1 in G if h != g1])])
    return G

In [49]:
#candidate elimination
domains = get_domains(examples)[:-1]

G = set([g_0(len(domains))])
S = set([s_0(len(domains))])
i=0
print(f'\n S[{i}]: {S}')
print(f' G[{i}]: {G}')

for xcx in examples:
    i=i+1
    xcx=list(xcx)

    x, cx = xcx[:-1], xcx[-1]  # Splitting data into attributes and decisions
    if cx=='Y': # x is positive example
        G = {g for g in G if fulfills(x, g)}
        S = generalize_S(x, G, S)
    else: # x is negative example
        S = {s for s in S if not fulfills(x, s)}
        G = specialize_G(x, domains, G, S)
    print(f'\n S[{i}]: {S}')
    print(f' G[{i}]: {G}')


 S[0]: {('0', '0', '0')}
 G[0]: {('?', '?', '?')}

 S[1]: {('0', '0', '0')}
 G[1]: {('?', '?', 'Triangle'), ('Small', '?', '?'), ('?', 'Blue', '?')}

 S[2]: {('0', '0', '0')}
 G[2]: {('?', 'Blue', '?'), ('Big', '?', 'Triangle'), ('Small', '?', 'Circle')}

 S[3]: {('Small', 'Red', 'Circle')}
 G[3]: {('Small', '?', 'Circle')}

 S[4]: {('Small', 'Red', 'Circle')}
 G[4]: {('Small', '?', 'Circle')}

 S[5]: {('Small', '?', 'Circle')}
 G[5]: {('Small', '?', 'Circle')}
